In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import xgboost 
from neupy.estimators import mse
from datetime import date
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dta = pd.read_excel('../Data/Seasonal.xlsx',index_col='Date')
# dta['Jan'] = dta['Month']==1
dta.head()

,Death,Death_rate,Incidence,Incidence_rate,forecast,Year,Month,error_ARIMA
Date,,,,,,,,
2005-01-01,143,0.011001,114194,8.784966,NaN,2005,1,NaN
2005-02-01,201,0.015463,87112,6.701542,NaN,2005,2,NaN
2005-03-01,281,0.021617,130134,10.011233,NaN,2005,3,NaN
2005-04-01,374,0.028772,133924,10.302798,NaN,2005,4,NaN
2005-05-01,388,0.029849,121326,9.333632,NaN,2005,5,NaN


In [3]:
# dataset = dta.loc['2006-02-1':'2014-12-01'][['Incidence_rate','forecast','Month']]
# x_train = dataset.loc[:'2012-12-01'][['forecast','Month']]
# y_train = dataset.loc[:'2012-12-01']['Incidence_rate']
# x_test = dataset.loc['2013-01-01':'2013-12-01'][['forecast','Month']]
# y_test = dataset['2013-01-01':'2013-12-01']['Incidence_rate']

In [11]:
dataset = dta.loc['2006-02-1':'2014-12-01'][['Incidence_rate','forecast']]
x_train = dataset.loc[:'2012-12-01'][['forecast']]
y_train = dataset.loc[:'2012-12-01']['Incidence_rate']
x_test = dataset.loc['2013-01-01':'2013-12-01'][['forecast']]
y_test = dataset['2013-01-01':'2013-12-01']['Incidence_rate']

In [12]:
# fit model no training data
model = xgboost.XGBRegressor(objective='reg:linear')
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [13]:
def try_std(n_estimators=20,max_depth=5,learning_rate=0.1):
    model = xgboost.XGBRegressor(objective='reg:linear',\
                                 n_estimators=n_estimators,\
                                 max_depth=max_depth,\
                                 learning_rate=learning_rate)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(mse(y_pred,y_test))

In [21]:
for n_estimators in range(20,40,10):
    for max_depth in range(5,30,5):
        for learning_rate in range(10,20,1):
            print(n_estimators,max_depth,learning_rate)
            try_std(n_estimators=n_estimators,max_depth=max_depth,learning_rate=learning_rate/100)
            print('--\n')

20 5 10
0.4578457820302224
--

20 5 11
0.2999743686122562
--

20 5 12
0.21334364342538828
--

20 5 13
0.15253913419999413
--

20 5 14
0.10911851539624319
--

20 5 15
0.08073976115210679
--

20 5 16
0.06683116710161184
--

20 5 17
0.05275614054684615
--

20 5 18
0.04821134690488436
--

20 5 19
0.0449733268147089
--

20 10 10
0.4578457820302224
--

20 10 11
0.3000383108205808
--

20 10 12
0.21334364342538828
--

20 10 13
0.15213126856005285
--

20 10 14
0.11328561938549836
--

20 10 15
0.07932274901868706
--

20 10 16
0.0674105271166958
--

20 10 17
0.0542256509677962
--

20 10 18
0.049287594243478554
--

20 10 19
0.04706428900988303
--

20 15 10
0.4578457820302224
--

20 15 11
0.3000383108205808
--

20 15 12
0.21334364342538828
--

20 15 13
0.15213126856005285
--

20 15 14
0.11328561938549836
--

20 15 15
0.07932274901868706
--

20 15 16
0.0674105271166958
--

20 15 17
0.0542256509677962
--

20 15 18
0.049287594243478554
--

20 15 19
0.049301545161351844
--

20 20 10
0.4578457820302224


In [22]:
try_std(n_estimators=30,max_depth=5,learning_rate=0.18)

0.03467367947844699


In [23]:
# fit model no training data
model = xgboost.XGBRegressor(objective='reg:linear',n_estimators=20,max_depth=10,learning_rate=0.17)
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.17, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=20, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [24]:
dataset['XGBoost'] = model.predict(dataset[['forecast']])

In [25]:
mse(dataset['XGBoost'],dataset['Incidence_rate'])

0.11034602907604486

In [26]:
dataset.to_excel('../Data/Seasonal_XGBoost.xlsx')